In [18]:
import os
import geopandas as gpd
import json

from pathlib import Path
from traitlets import (
    Unicode, link, observe
)
import ipyvuetify as v
import sepal_ui.sepalwidgets as sw


In [31]:
from sepal_ui.scripts import utils as su
from sepal_ui.mapping import mapping as m

In [94]:
class AoiView(v.Card):
    
    method = Unicode('').tag(sync=True)
    country = Unicode('').tag(sync=True)
    file = Unicode('').tag(sync=True)
    column = Unicode('').tag(sync=True)
    field = Unicode('').tag(sync=True)

    def __init__(self, map_, *args, **kwargs):
        
        self.map_=map_
        self.methods = self.get_methods()
        
        w_method = v.Select(
            label = 'Select a method',
            v_model = self.method,
            items = self.get_methods()
        )
        
        w_countries = v.Select(
            label="Select country",
            v_model='',
            items=self.get_countries(),
        )
        btn_use_country = sw.Btn('Select country')
        
        w_column = v.Select(
            label="Filter by column",
            v_model=self.column,
            items=[]
        )
        
        w_field = v.Select(
            label="Select field",
            v_model=self.field,
            items=[]
        )
        
        w_shape_input = sw.FileInput(['.shp'], os.getcwd())
        btn_use_file = sw.Btn('Select country')
        
        self.components = {
            'Country' : self.w_countries,
            'Upload shapefile' : self.w_shape_input
        }
        
        self.hide_components()
        
        super().__init__(*args, **kwargs)
        
        
        # Link traits
        link((self, 'method'),(w_method, 'v_model'))
        link((self, 'country'),(w_countries, 'v_model'))
        link((self, 'file'),(w_shape_input, 'file'))
        link((self, 'column'),(w_column, 'v_model'))
        link((self, 'field'),(w_field, 'v_model'))
        
        # Events
        btn_use_country.on_event('click', self.add_filter_widget)
        btn_use_file.on_event('click', self.add_filter_widget)
        
        self.children=[
            w_method,
            self.w_countries,
            self.w_shape_input,
        ]
        
    def add_filter_widget(self):
        """Add column and field filter after selecting a dataset"""
        
        # Almost all datasets are composed by more than one feature(geometry)
        # this function will allow only select one (or more?) geometries.
        
        
        
    def hide_components(self):
        """Hide all possible componentes"""
        
        for component in self.components.values():
            su.hide_component(component)
        
    def get_methods(self):
        """Handle which methods are displayed in select widget"""
        
        return ['Draw on map', 'Country', 'Upload shapefile']
    
    def get_countries(self):
        """Create a list of countries"""
        
        return list(range(10))
    
    @observe('method')
    def aoi_method_event(self, change):
        
        method = change['new']
        self.hide_components()
        
        if method == 'Draw on map':
            self.map_.show_dc()
        else:
            self.map_.hide_dc()
            su.show_component(self.components[method])

In [95]:
w_shape_input = sw.FileInput(['.shp'], os.getcwd())

In [96]:
# map_=m.SepalMap()

In [97]:
# aoi_model = AoiModel()

In [98]:
AoiView(map_=map_)

AoiView(children=[Select(items=['Draw on map', 'Select country', 'Upload shapefile'], label='Select a method',…

In [16]:
class AoiModel:
    
    shapefile = Unicode('').tag(sync=True)
    
    def __init__(self, aoi, **kwargs):
    
        self.aoi = aoi
        self.gdf = None
        
        super().__init__(**kwargs)
        
        # Parameters
        self.out_path = Path('')
        self.json_path = None
        
        # Widgets
        
        self.w_state_bar = sw.StateBar(loading=False) if not statebar else statebar
        
        # Link behaviours 
        
        link((self.shape_input, 'file'), (self, 'shapefile'))
        
        # View
        
        self.children=[
            v.Layout(row=True, children=[
                self.shape_input,
            ])
        ]
        
        # Add a statebar if there is not provided an external one
        if not statebar: self.children = self.children + [self.w_state_bar]

    @observe('shapefile')
    def shape_to_geojson(self, change):
        """ Converts shapefile into Json file"""
        
        shp_file_path = Path(self.shapefile)
        if shp_file_path.suffix == '.shp':

            self.gdf = gpd.read_file(str(shp_file_path))
            self.gdf = self.gdf.to_crs("EPSG:4326")
            
            self.json_path = shp_file_path.parent/f'{shp_file_path.stem}.geojson'
            
            if not self.json_path.exists():
                self.w_state_bar.add_msg('Converting shape to GeoJSON', loading=False)
                self.gdf.to_file(str(self.json_path), driver='GeoJSON')
                self.w_state_bar.add_msg('Done', loading=True)
            else:
                self.w_state_bar.add_msg('Geojson file already created', loading=False)
                
    def get_ipyleaflet_geojson(self):
        """Returns GeoJSON ipyleaflet object from Json file"""
        
        if self.json_path:
            self.w_state_bar.add_msg('Converting shape to GeoJSON', loading=False)
            with open(self.json_path) as f:
                data = json.load(f)        
                ipygeojson = GeoJSON(
                    data=data,
                    name=self.json_path.stem, 
                    style={'color': 'green', 'fillOpacity': 0, 'weight': 3})

            self.w_state_bar.add_msg('Done', loading=True)
        
            return ipygeojson
        else:
            self.w_state_bar.add_msg('There is not a shapefile selected.', loading=False)

In [17]:
AOI()

AOI(children=[Layout(children=[FileInput(children=[Btn(children=[Icon(children=['mdi-cached'])], color='primar…